# **과제2 : 챗봇 만들기2**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기2
    * Vector DB에 데이터 추가하기
    * Retriever, memory, LLM를 연결하기
    * 실행시 이력 DB 생성하고 기록하기
    * test

## **1.환경준비**

### (1) 라이브러리 Import

In [14]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [15]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

sk-proj-TTDjUJt5vPHm3UKT7LjyT3BlbkFJM02q2MKo0P1KaTDxyISl


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [62]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩 
    * 1일차에서 제공한 csv 파일의 구조를 그대로 이용
    * 에이블스쿨 홈페이지 FAQ 데이터 수집(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문을 csv 형태로 저장
    * 데이터프레임으로 저장하기

In [3]:
import requests
from bs4 import BeautifulSoup
import json

headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding' : 'gzip, deflate, br, zstd',
    'Accept-Language' : 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cache-Control' : 'no-cache',
    'Connection' : 'keep-alive',
    'Cookie' : 'SESSION=2c98596b-6556-4a19-a33b-9916f6c6c292; TS01e6903a=01a4caa0ef0a73160fe26afb501c63a9062d593353a5131d9feecf618cd65395e3e03f717e76c3342ffb87c81548c61d1bb893457a2531a02d29e95e391c7493d391dd60aa',
    'Host' : 'aivle.kt.co.kr',
    'Pragma' : 'no-cache',
    'Referer' : 'https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056',
    'Sec-Ch-Ua' : '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
    'Sec-Ch-Ua-Mobile' : '?0',
    'Sec-Ch-Ua-Platform' : '"Windows"',
    'Sec-Fetch-Dest' : 'empty',
    'Sec-Fetch-Mode' : 'cors',
    'Sec-Fetch-Site' : 'same-origin',
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
    'X-Requested-With' : 'XMLHttpRequest',

}

qa_data = {'question':[], 'category':[], 'answer':[]}

# 혹시 response 500 코드 반환시 'Cookie'의 값을 크롬에서 다시 가져오기
for i in range(1,8):
    url = f'https://aivle.kt.co.kr/home/brd/faq/listJson?ctgrCd=&pageIndex={i}'
    response = requests.get(url, headers=headers)
    data = response.json()
    for k in range(10):
        try:
            qa_data['question'].append(data['returnList'][k]['atclTitle'])
            qa_data['category'].append(data['returnList'][k]['ctgrNm'])
            soup = BeautifulSoup(data['returnList'][k]['atclCts'], 'html.parser') # HTML 파싱
            text_only = soup.get_text() # 텍스트만 추출
            qa_data['answer'].append(text_only)
        except:
            continue

C:\Users\User\AppData\Local\Temp\ipykernel_8080\3725768748.py:37: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(data['returnList'][k]['atclCts'], 'html.parser') # HTML 파싱


In [4]:
qa_data

{'question': ['1) 최종 학력 또는 전공과 관계없이 지원할 수 있나요? ',
  '2) 35세 이상은 지원할 수 없나요?',
  '3) 미취업자의 기준이 뭔가요?',
  '4) 직장인도 지원할 수 있나요? ',
  '5) 아르바이트를 하고 있는데 지원할 수 있나요? ',
  '6) 사업자도 지원할 수 있나요? ',
  '7) 프리랜서도 지원할 수 있나요? ',
  '8) 현재 군 복무중인 상태인데, 지원이 가능한가요?',
  '9) 퇴사 후 실업급여 수령 중인데, 지원할 수 있나요? ',
  '10) 다른 K-DT 과정을 들었는데 지원할 수 있나요?',
  '11) K-DT가 아닌 국비과정 참여 중인데 지원할 수 있나요?',
  '12) 이전에 불합격했을 경우 이번에 다시 지원할수 있나요?',
  '13) 합격 후 과정을 포기하면 재지원이 불가능한가요?',
  '14) 추가합격제도가 있나요?',
  '15) 면접은 어떻게 진행되나요?',
  '16) 코딩테스트는 어떻게 진행되나요?',
  '17) 국민내일배움카드가 있어야 인적성검사 응시할 수 있나요? ',
  '18) 인적성검사는 KT 채용과 같은 과목인가요? ',
  '19) 국민내일배움카드가 있어야만 지원할 수 있나요? ',
  '20) (지원서 작성) 현재 4학년 1학기 재학 또는 휴학 등의 사유로 졸업(예정)증명서 발급이 안되는 경우 어떻게 해야 하나요?',
  '21) (지원서 작성) 대학원 재학 중으로 성적표가 없는 경우 어떻게 하나요?',
  '22) (지원서 작성) 미국 4년제 대학의 경우 증빙서류가 무엇인가요? ',
  '23) 컴퓨터공학 비전공자도 커리큘럼을 따라갈 수 있을까요?',
  '24) 교육 전에 미리 공부하면 좋은 것이 있을까요?',
  '25) 출결 기준이 어떻게 될까요?',
  '26) 휴가를 사용할 수 있나요?',
  '27) 교육 수료 기준은 무엇인가요?',
  '28) 교육 장소(지역)는 어떻게 되나요?',
  '29) 지원 지역(교육 장소)을 변경할 수 있나

In [5]:
aivle_qa = {'구분':[], 'QA':[]}

for i in range(len(qa_data['question'])):
    aivle_qa['구분'].append(qa_data['category'][i])
    q = qa_data['question'][i]
    q = q[(q.find(')')+2):] # 맨앞의 숫자 제거
    aivle_qa['QA'].append('"' + q + '\n' + qa_data['answer'][i] + '"')

aivle_qa_df = pd.DataFrame(aivle_qa)
aivle_qa_df

,구분,QA
0,모집/선발,"""최종 학력 또는 전공과 관계없이 지원할 수 있나요? \nKT 에이블스쿨은 정규 4..."
1,모집/선발,"""35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 ..."
2,모집/선발,"""미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다. \n1) 기간의 ..."
3,모집/선발,"""직장인도 지원할 수 있나요? \nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육..."
4,모집/선발,"""아르바이트를 하고 있는데 지원할 수 있나요? \n고용보험에 가입이 되어 있는 경우..."
...,...,...
63,기타,"""AICE (기존 AIFB) 자격증이 있어야 유리한가요?\n네. AICE (기존 A..."
64,기타,"""AICE 시험관련 문의는 어디에 해야하나요? \nhelp@aice.study 으로..."
65,기타,"""국민취업지원제도를 신청하고 싶습니다. \n국민취업지원제도 신청과 관련 사항은 거주..."
66,기타,"""국민취업지원제도를 안해도 교육 참여되나요? \n국민취업제도를 신청하지 않아도 KT..."


In [6]:
aivle_qa_df.to_csv('aivle_qa_df.csv',index=False)

* 벡터 데이터베이스
    * 1일차 벡터 데이터베이스를 그대로 이용
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./db



In [3]:
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory='./db', # 경로지정
                  embedding_function = embedding, # 임베딩 백터로 만들 모델 지정
                  )

* 데이터 입력
    * 기존 입력을 모두 제거하고 추가 사항만 모두 입력
    * meta data로 '구분' 칼럼 값 추가하기

In [8]:
data = pd.read_csv('aivle_qa_df.csv', encoding='utf-8')
data.head()

,구분,QA
0,모집/선발,"""최종 학력 또는 전공과 관계없이 지원할 수 있나요? \nKT 에이블스쿨은 정규 4..."
1,모집/선발,"""35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 ..."
2,모집/선발,"""미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다. \n1) 기간의 ..."
3,모집/선발,"""직장인도 지원할 수 있나요? \nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육..."
4,모집/선발,"""아르바이트를 하고 있는데 지원할 수 있나요? \n고용보험에 가입이 되어 있는 경우..."


In [9]:
# 데이터프레임의 데이터를 list로 변환
qa_list = data['QA'].tolist()
cat_list = data['구분'].tolist()

# 리스트 내용을 각각 document로 변환
document = [Document(page_content=qa_list[i], metadata={'category' : cat_list[i]}) for i in range(len(qa_list))]

ind = database.add_documents(document)

database.get(ind)

{'ids': ['00daf515-5b6d-4449-a549-8039a5929daf',
  '08d30775-cc10-4ac6-9cbf-67ecba6acffa',
  '162f2d1c-9c53-43c9-921e-d4cf1ebf4bd4',
  '18663574-b797-40fa-9fe0-a54cf0bb8ddf',
  '188d47e8-721a-4cb6-919c-8307dfec0674',
  '24ec4a5f-25ea-4ec6-9999-411c9a4678a8',
  '25c417a6-9e32-48b3-9833-a9bbcfff56d4',
  '272fe075-d806-46c1-bb3d-702fdfcd6b05',
  '28fd76e4-84bf-44ef-a1be-628493155962',
  '3e7e180e-8514-48e0-beb0-51e212174926',
  '3e8bb701-3c26-4a9e-a1a4-60bf3a27927c',
  '417915a5-4c32-4b4b-b35a-205a7a0cfa87',
  '46e733ec-4e4d-424b-a54d-8f9b2e7752a9',
  '4e51b2b6-f312-449d-9bdb-c08e4942fc4f',
  '511216f9-8425-4563-bad6-2cc705ff9bfb',
  '52059d89-3ede-483c-9411-f49a1aa4f148',
  '54bddc69-edab-44b9-9f1f-66f76e8e053c',
  '59193ce8-7e9c-474f-be71-5e7046dbf58d',
  '5a1c0c9f-98cb-45ce-a181-85b67c625092',
  '5ae15774-ea1f-4191-9be4-c4e7c273ebe9',
  '5b03c931-0d6b-475b-bfdb-51cedb344db7',
  '5f5b445f-61a6-47a8-8b12-46390c95eab3',
  '5fb2a9e8-a54e-47e8-b97c-64e571faa787',
  '62ade42d-ec77-4f43-9419-

* 입력된 데이터 조회

In [10]:
database.get()

{'ids': ['00daf515-5b6d-4449-a549-8039a5929daf',
  '02bdf0fa-fb5a-4c00-93cd-f9db3494c526',
  '02df3534-2bbe-4584-a4e6-d8761cdc39af',
  '05e3ffbd-3008-4ef2-858e-64b36f97a09c',
  '08237063-ec2d-49b1-bd28-812011b005e7',
  '08d30775-cc10-4ac6-9cbf-67ecba6acffa',
  '0cc0c8e0-604d-41d3-bfb9-ddb2373cd81e',
  '0d01bcd6-eae3-4ce4-a00a-38c08d224345',
  '0f34bd5f-6a4d-41f6-8380-b60cf769ea6f',
  '1443a6ad-4c47-4a68-af30-6dc4b1c2cbd5',
  '156a6b55-536d-4251-bbad-455381cf4d51',
  '162f2d1c-9c53-43c9-921e-d4cf1ebf4bd4',
  '18663574-b797-40fa-9fe0-a54cf0bb8ddf',
  '188d47e8-721a-4cb6-919c-8307dfec0674',
  '1f8aed0d-58db-4a4f-8c22-483f07bb1f07',
  '1fc7ec76-88d5-4de3-8d83-cdefab50f7b0',
  '227ee59a-7c3d-4d48-b9e8-3b68f4eeec14',
  '231d6e96-a3ea-47e9-b5b6-eb9f3611d3b2',
  '24ec4a5f-25ea-4ec6-9999-411c9a4678a8',
  '25c417a6-9e32-48b3-9833-a9bbcfff56d4',
  '26351a0f-0234-42ea-9f66-04f038aab63a',
  '272fe075-d806-46c1-bb3d-702fdfcd6b05',
  '28fd76e4-84bf-44ef-a1be-628493155962',
  '2b69135f-125e-4c31-ab01-

## **3.RAG+LLM모델**

* 모델 : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정
    * memory 사용
* 요구사항
    * 질문 history 관리를 위한 이력 저장 DB 생성
        * DB 명 : db_chatlog
        * 테이블 명 : history
            * id INTEGER PRIMARY KEY : 이렇게 설정하면 자동증가 값으로 채워짐
            * datetime TEXT : 질문시점 yyyy-mm-dd hh:mi:ss
            * query TEXT : 질문
            * sim1 REAL : 첫번째 문서의 유사도 점수
            * sim2 REAL : 두번째 문서의 유사도 점수
            * sim3 REAL : 세번째 문서의 유사도 점수
            * answer TEXT : 답변
        * 유사도 점수는 similarity_search_with_score 메서드를 이용해서 저장해야 함
        * 질문과 답변이 진행될 때마다 history 테이블에 데이터 입력

* 관리용 DB, 테이블 생성

In [19]:
path = './db_chatlog/db_chatlog.db'
conn = sqlite3.connect(path)

In [20]:
# 커서 객체 생성
cursor = conn.cursor()

# test 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS history (
    id INTEGER PRIMARY KEY,
    datetime TEXT NOT NULL,
    query TEXT NOT NULL,
    sim1 REAL,
    sim2 REAL,
    sim3 REAL,
    answer TEXT
)
''')

# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

* 모델 선언

In [21]:
database = Chroma(persist_directory='./db', # 경로지정
                  embedding_function = embedding, # 임베딩 백터로 만들 모델 지정
                  )

embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = "./db", embedding_function = embeddings)
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=3
retriever = database.as_retriever(search_kwargs={"k": k})

# 대화 메모리 생성(메모리 초기화)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True)

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")

* 모델 사용 및 이력 확인

In [22]:
from datetime import datetime

def aivle_gpt(query):
    result = qa(query)
    print(f'질문 : {query}')
    print('답변 :', result['answer'])
    # 입력할 데이터
    dt = datetime.now()
    sims = database.similarity_search_with_score(query, k = 3)
    
    data = pd.DataFrame({'datetime' : [dt], 
                         'query' : [query],
                         'sim1' : [sims[0][1]],
                         'sim2' : [sims[1][1]],
                         'sim3' : [sims[2][1]],
                         'answer' : [result['answer']]
                         })
    # db에 저장
    conn = sqlite3.connect(path)
    data.to_sql('history', conn, if_exists='append', index=False) # test 테이블이 있으면 insert, 없으면 생성
    conn.close()
    

In [27]:
# 첫번째 질문
query = '에이블스쿨이 뭐야?'
aivle_gpt(query)

질문 : 에이블스쿨이 뭐야?
답변 : 에이블스쿨은 KT와 정부가 함께 하는 프로그램으로, 미래인재육성 및 청년 IT/SW/DX 분야 일자리 창출을 위해 운영되는 프로그램입니다. 에이블러들을 위해 취업기회를 제공하기 위해 Job Searching 세션, 취업컨설팅, 잡페어 등 다양한 프로그램을 운영하고 있습니다.


In [28]:
# 두번째 질문
query = '내일배움카드가 뭐야?'
aivle_gpt(query)

질문 : 내일배움카드가 뭐야?
답변 : 네, 정확히 맞습니다. 국민내일배움카드는 국가에서 국민이 스스로 직업능력개발훈련을 실시할 수 있도록 훈련비를 지원하는 제도입니다.


In [29]:
# 세번째 질문
query = '이것만 있으면 에이블스쿨을 수강할 수 있어?'
aivle_gpt(query)

질문 : 이것만 있으면 에이블스쿨을 수강할 수 있어?
답변 : 에이블스쿨을 수강하기 위해서는 일반적으로 관련된 자격 요건이나 등록 절차가 필요할 수 있습니다. 또한, 특정 교육 과정에 따라 추가적인 요구 사항이 있을 수 있습니다. 자세한 내용은 해당 교육 프로그램의 홈페이지나 담당 기관에 문의하시는 것이 좋습니다.


In [30]:
# 네번째 질문
query = '수강하고 나면 취업이 될까?'
aivle_gpt(query)

질문 : 수강하고 나면 취업이 될까?
답변 : 취업 여부는 교육을 마친 후 채용 연계 시 참가한 기업의 판단에 따라 결정됩니다. 졸업자로 인정되지 않을 수 있으므로 해당 시점에 채용공고를 확인하시고, 기업의 요구사항을 충족시키는 것이 중요합니다.


In [12]:
# 메모리 확인
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='에이블스쿨이 뭐야?'),
  AIMessage(content='에이블스쿨은 KT와 정부가 함께하는 프로그램으로, 미래인재육성 및 청년 IT/SW/DX 분야 일자리 창출을 위해 운영되는 교육 프로그램입니다. 면접 과정은 따로 없으며, 취업 연계 지원을 위해 다양한 프로그램을 운영하고 있습니다.'),
  HumanMessage(content='내일배움카드가 뭐야?'),
  AIMessage(content='국민내일배움카드는 국가에서 국민 스스로 직업능력개발훈련을 실시할 수 있도록 훈련비를 지원하는 제도입니다. 이 카드를 통해 1인당 300만 원에서 500만 원까지 지원을 받을 수 있습니다.'),
  HumanMessage(content='이것만 있으면 에이블스쿨을 수강할 수 있어?'),
  AIMessage(content='아닙니다. 국민내일배움카드 발급이 가능한 만 34세 이하의 정규 4년제 대학(전공무관) 졸업자 또는 졸업 예정자 중에 해당하는 경우에만 KT 에이블스쿨 훈련과정을 들을 수 있습니다.'),
  HumanMessage(content='수강하고 나면 취업이 될까?'),
  AIMessage(content='KT 에이블스쿨은 취업을 지원하는 교육이며, 채용 연계가 있습니다. 그러나 졸업이 유예된 경우 채용이 어려울 수 있으므로 채용 공고를 확인하고 기업의 정책을 따로 확인해야 합니다.')]}

In [31]:
# db 확인
conn = sqlite3.connect(path)
df = pd.read_sql('SELECT * FROM history', conn)
display(df)
conn.close()

,id,datetime,query,sim1,sim2,sim3,answer
0,1,2024-06-04 16:04:48.429032,에이블스쿨이 뭐야?,0.329628,0.330200,0.344886,"에이블스쿨은 KT와 정부가 함께 하는 프로그램으로, 미래인재육성 및 청년 IT/SW..."
1,2,2024-06-04 16:05:14.352958,내일배움카드가 뭐야?,0.250343,0.250363,0.268401,국민내일배움카드는 국가에서 국민 스스로 직업능력개발훈련을 실시할 수 있도록 훈련비를...
2,3,2024-06-04 16:05:18.490055,이것만 있으면 에이블스쿨을 수강할 수 있어?,0.277711,0.278031,0.284442,"에이블스쿨을 수강하기 위해서는 온라인으로 신청을 하고, 필수 참여 시간에 맞춰 수업..."
3,4,2024-06-04 16:05:22.850477,수강하고 나면 취업이 될까?,0.233308,0.233351,0.273511,KT 에이블스쿨에서는 교육 수료 후 취업이 가능합니다. 하지만 교육 시작일 기준 재...
4,5,2024-06-04 16:05:43.810171,에이블스쿨이 뭐야?,0.329628,0.330200,0.344886,"에이블스쿨은 KT와 정부가 함께 하는 프로그램으로, 미래인재육성 및 청년 IT/SW..."
5,6,2024-06-04 16:05:47.971497,내일배움카드가 뭐야?,0.250343,0.250363,0.268401,"네, 정확히 맞습니다. 국민내일배움카드는 국가에서 국민이 스스로 직업능력개발훈련을 ..."
6,7,2024-06-04 16:05:54.149493,이것만 있으면 에이블스쿨을 수강할 수 있어?,0.277711,0.278031,0.284442,에이블스쿨을 수강하기 위해서는 일반적으로 관련된 자격 요건이나 등록 절차가 필요할 ...
7,8,2024-06-04 16:05:58.040573,수강하고 나면 취업이 될까?,0.233308,0.233351,0.273511,취업 여부는 교육을 마친 후 채용 연계 시 참가한 기업의 판단에 따라 결정됩니다. ...
